In [1]:
!python -m spacy download en_core_web_sm

2023-11-17 13:14:15.685828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 13:14:15.829933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-17 13:14:15.830002: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-17 13:14:15.861155: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 13:14:17.951949: W tensorflow/stream_executor/platform/de

In [2]:
!pip install xgboost==2.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 6.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install catboost==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 14.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install optuna==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 83.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

#NLP libraries
import re
import spacy
import string
from sklearn.feature_extraction.text import TfidfVectorizer


#ML and models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

import pickle

import optuna
from sklearn.model_selection import GridSearchCV

# to ignore all messages of algorythm deprecation
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


from tabulate import tabulate 


/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-17 13:14:48.406175: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 13:14:48.548718: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-17 13:14:48.548771: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-17 

## Import of dataset and first investigation

In [69]:
df = pd.read_csv("youtoxic_english_1000.csv")
df.head()

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


In [7]:
df.shape

(1000, 15)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CommentId        1000 non-null   object
 1   VideoId          1000 non-null   object
 2   Text             1000 non-null   object
 3   IsToxic          1000 non-null   bool  
 4   IsAbusive        1000 non-null   bool  
 5   IsThreat         1000 non-null   bool  
 6   IsProvocative    1000 non-null   bool  
 7   IsObscene        1000 non-null   bool  
 8   IsHatespeech     1000 non-null   bool  
 9   IsRacist         1000 non-null   bool  
 10  IsNationalist    1000 non-null   bool  
 11  IsSexist         1000 non-null   bool  
 12  IsHomophobic     1000 non-null   bool  
 13  IsReligiousHate  1000 non-null   bool  
 14  IsRadicalism     1000 non-null   bool  
dtypes: bool(12), object(3)
memory usage: 35.3+ KB


In [9]:
df.isnull().sum()

CommentId          0
VideoId            0
Text               0
IsToxic            0
IsAbusive          0
IsThreat           0
IsProvocative      0
IsObscene          0
IsHatespeech       0
IsRacist           0
IsNationalist      0
IsSexist           0
IsHomophobic       0
IsReligiousHate    0
IsRadicalism       0
dtype: int64

In [10]:
df.nunique()

CommentId          1000
VideoId              13
Text                997
IsToxic               2
IsAbusive             2
IsThreat              2
IsProvocative         2
IsObscene             2
IsHatespeech          2
IsRacist              2
IsNationalist         2
IsSexist              2
IsHomophobic          1
IsReligiousHate       2
IsRadicalism          1
dtype: int64

In [11]:
col = list(df.columns)
for col_name in col:
  print(f'Name: {col_name}')
  print(df[col_name].value_counts())

Name: CommentId
Ugj5MKvyX16BlngCoAEC          1
Ugg97M7HImLjcngCoAEC          1
UggwTBfPMkN0gngCoAEC          1
Ugin-dBfgSlydXgCoAEC          1
Ugz8uVdEHIiNFvrakgV4AaABAg    1
                             ..
UggJNJo4salVHXgCoAEC          1
Ugi_4iRsu9Skx3gCoAEC          1
Ugy0IpiM6lrJHHQilvh4AaABAg    1
UghcJ70N2jvmUXgCoAEC          1
UgjwWrcF5iU6s3gCoAEC          1
Name: CommentId, Length: 1000, dtype: int64
Name: VideoId
9pr1oE34bIM    274
04kJtp6pVXI    172
cT14IbTDW2c    146
dG7mZQvaQDk    112
TZxEyoplYbI    104
bUgKZMSxr3E     39
8HB18hZrhXc     38
4rCweDxDqdw     37
Dt9-byUhPdg     34
#NAME?          16
XRuCW80L9mA     10
dDbRyFIkNII     10
5vF4si3hoRA      8
Name: VideoId, dtype: int64
Name: Text
run them over                                                                                                                                                                                                                                                                                   

In [12]:
duplicates = df[df.duplicated('Text', keep=False)]
duplicate_values = duplicates['Text'].tolist()
print(duplicate_values)


['RUN THEM OVER', 'run them over', 'run them over', 'run them over', 'RUN THEM OVER']


## Conversion from object to int and creation new summarising column hate or not

In [70]:
bool_col = df.select_dtypes(include = ['bool'])
# Convert boolean columns to integers
df[bool_col.columns] = df[bool_col.columns].astype(int)
df.head()

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,0,0,0,0,0,0,0,0,0,0,0,0
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,1,1,0,0,0,0,0,0,0,0,0,0
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,1,1,0,0,1,0,0,0,0,0,0,0
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,0,0,0,0,0,0,0,0,0,0,0,0
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",0,0,0,0,0,0,0,0,0,0,0,0


In [71]:
df_sumhate = df.copy()
df_sumhate.shape

(1000, 15)

In [72]:
df_sumhate['Hate'] = df_sumhate.select_dtypes( include= ['int']).any(axis=1).astype(int)
df_sumhate.head()

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism,Hate
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,1,1,0,0,0,0,0,0,0,0,0,0,1
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,1,1,0,0,1,0,0,0,0,0,0,0,1
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",0,0,0,0,0,0,0,0,0,0,0,0,0


### Investigate targeted variable 

Looking at different "kind of hate" comments distribution some of them looks very unbalanced but if we look at resumen "hate" column it looks quite balanced

## Text cleaning with spacy

In [16]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [73]:

nlp = spacy.load('en_core_web_sm')

In [74]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS
print(len(stopwords))

326


In [75]:

def clean_text(text):
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # eliminate URL
    text = re.sub(r'@\w+', '', text)  # eliminate  mentions
    text = re.sub(r'#\w+', '', text)  # eliminate  hashtags
    text = re.sub(r'[^\x00-\x7F]+', '', text)  #specific symbols which are not in ASCII
    text = re.sub(r'\d+', '', text)  # eliminate numbers
    tokens = nlp(text)  # tokenization with spacy
    text = " ".join([word.lemma_ for word in tokens if word.lemma_ not in stopwords])  #checking stopwords plus lematization
    text = re.sub(r'\s+', ' ', text).strip() # eliminate extra space
    text = re.sub(r'[^\w\s]', '', text)  # eliminate punctuation

    return text.lower() 


df_sumhate['processed_text'] = df_sumhate['Text'].apply(lambda x: clean_text(x))
df_sumhate.head()


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism,Hate,processed_text
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,0,0,0,0,0,0,0,0,0,0,0,0,0,people step case people situation lump mess ...
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,1,1,0,0,0,0,0,0,0,0,0,0,1,law enforcement train shoot apprehend train s...
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,1,1,0,0,1,0,0,0,0,0,0,0,1,reckon black life matter banner hold white c...
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,0,0,0,0,0,0,0,0,0,0,0,0,0,large number people like police officer crimi...
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",0,0,0,0,0,0,0,0,0,0,0,0,0,arab dude absolutely right shoot extra time ...


In [76]:
df_final= df_sumhate[['processed_text', 'Hate']]
df_final.head()

,processed_text,Hate
0,people step case people situation lump mess ...,0
1,law enforcement train shoot apprehend train s...,1
2,reckon black life matter banner hold white c...,1
3,large number people like police officer crimi...,0
4,arab dude absolutely right shoot extra time ...,0


## Vectorisation with TF-IDF 

In [77]:
tfv = TfidfVectorizer(ngram_range=(1, 2),max_df=0.7, min_df=3, max_features=5000)

X_tfv =  tfv.fit_transform(df_final['processed_text'])
X_tfv.shape

(1000, 1160)

In [78]:
print(tfv.get_feature_names_out())

['ability' 'able' 'absolutely' ... 'youtube' 'zimmerman' 'zimmerman case']


In [79]:
X_features = pd.DataFrame(X_tfv.toarray())
X_features.columns = tfv.get_feature_names_out()
y = df_final['Hate']
X_features.head()


,ability,able,absolutely,abuse,accord,account,accountable,act,act like,action,...,yea,yeah,year,year old,yes,young,young man,youtube,zimmerman,zimmerman case
0,0.0,0.0,0.00000,0.0,0.0,0.0,0.124136,0.0,0.0,0.209767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.21855,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Holdout set

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2)

In [81]:
X_train1=X_train.copy()
y_train1=y_train.copy()

## Preprocessor

In [82]:
#creating transformer from cleaning function
func_trans_clean = FunctionTransformer(clean_text)

In [83]:
preprocessor = Pipeline([('clean', func_trans_clean),('tfv', TfidfVectorizer())])

## Model selection

In [28]:
models = [
    RandomForestClassifier(n_estimators=50, random_state= 42), 
    LogisticRegression(random_state =42, penalty="l2",C=0.01), 
    GradientBoostingClassifier(n_estimators=50, random_state =42),
    XGBClassifier(random_state =42),
    CatBoostClassifier(random_state =42),
    LinearSVC(random_state=42, dual=False, max_iter=100, C=0.01, penalty='l2', loss='squared_hinge')
]

In [29]:
def evaluate_models(models, X_train, y_train):
    skf = StratifiedKFold(n_splits=5)

    
    results = []

    for i, model in enumerate(models):
        model.fit(X_train, y_train)
        scores = cross_val_score(model, X_train, y_train.values.ravel(), cv=skf)

        y_pred = model.predict(X_test)
        accuracy = round(accuracy_score(y_test, y_pred), 3)
        scores = score(y_test, y_pred)
        mean_recall = round(scores[1].mean(), 3)
        mean_precision = round(scores[0].mean(), 3)
        mean_f1 = round(scores[2].mean(),3)

        train_pred = model.predict(X_train1)
        acc_train = accuracy_score(y_train1,train_pred)
        overfitting = (acc_train-accuracy)*100 

        #overfit_acc = ((scores['train_accuracy']) - accuracy) * 100
        # Append the results
        model_metrics = {'Model': model,
                        'Model score': scores,
                        'Accuracy': accuracy, 
                        'Precision': mean_precision,
                        'Recall': mean_recall,
                        'F1': mean_f1,
                        'Overfit%': overfitting}
        results.append(model_metrics)
          
     # Create a DataFrame from the results list
    results_df = pd.DataFrame(results)

    return results_df

In [30]:
results_df= evaluate_models(models, X_train1, y_train1)

1:	learn: 0.6902011	total: 14.9ms	remaining: 7.43s
2:	learn: 0.6888344	total: 21.6ms	remaining: 7.18s
3:	learn: 0.6874821	total: 28.6ms	remaining: 7.13s
4:	learn: 0.6862766	total: 42ms	remaining: 8.36s
5:	learn: 0.6848108	total: 49.6ms	remaining: 8.22s
6:	learn: 0.6836987	total: 57.3ms	remaining: 8.13s
7:	learn: 0.6826622	total: 64.1ms	remaining: 7.94s
8:	learn: 0.6817650	total: 70.8ms	remaining: 7.79s
9:	learn: 0.6807263	total: 77.5ms	remaining: 7.67s
10:	learn: 0.6794219	total: 84.4ms	remaining: 7.58s
11:	learn: 0.6782957	total: 91.1ms	remaining: 7.5s
12:	learn: 0.6769316	total: 98.2ms	remaining: 7.46s
13:	learn: 0.6757154	total: 105ms	remaining: 7.4s
14:	learn: 0.6746080	total: 112ms	remaining: 7.34s
15:	learn: 0.6729563	total: 119ms	remaining: 7.29s
16:	learn: 0.6714854	total: 125ms	remaining: 7.24s
17:	learn: 0.6705278	total: 132ms	remaining: 7.22s
18:	learn: 0.6692682	total: 139ms	remaining: 7.2s
19:	learn: 0.6683596	total: 147ms	remaining: 7.21s
20:	learn: 0.6675077	total: 155ms

In [31]:
results_df

,Model,Model score,Accuracy,Precision,Recall,F1,Overfit%
0,"(DecisionTreeClassifier(max_features='sqrt', r...","([0.6693548387096774, 0.7763157894736842], [0....",0.710,0.723,0.710,0.706,28.375
1,"LogisticRegression(C=0.01, random_state=42)","([0.5, 0.0], [1.0, 0.0], [0.6666666666666666, ...",0.500,0.250,0.500,0.333,4.750
2,([DecisionTreeRegressor(criterion='friedman_ms...,"([0.5973154362416108, 0.7843137254901961], [0....",0.645,0.691,0.645,0.622,16.875
3,"XGBClassifier(base_score=None, booster=None, c...","([0.6610169491525424, 0.7317073170731707], [0....",0.690,0.696,0.690,0.687,24.875
4,<catboost.core.CatBoostClassifier object at 0x...,"([0.6194029850746269, 0.7424242424242424], [0....",0.660,0.681,0.660,0.650,24.500
5,"LinearSVC(C=0.01, dual=False, max_iter=100, ra...","([0.5240641711229946, 0.8461538461538461], [0....",0.545,0.685,0.545,0.439,7.875


Based on first model check the best results have RF, XGBC, Catboost and GBC but the overfitting is significat, so going to follow with parameter adjustment for selected models with Gridsearch or Optuna

### 

## Hyperparameters selection for  RF, XGBC, Catboost and GBC with  Gridsearch and Optuna

In [32]:
# train the model
rf = RandomForestClassifier(random_state= 42)
xgb = XGBClassifier(random_state =42)
cat =CatBoostClassifier(random_state =42)
gbc = GradientBoostingClassifier(random_state =42)


### GradientBoostingClassifier simple  cycle

In [33]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, train_support = score(y_test, y_pred, pos_label=1, average='binary')
    print('Est: {} / Depth: {} / LR: {} ----- Precision: {} / Recall: {} / Accuracy: {}'.format(
            est, max_depth, lr, round(precision, 3), round(recall, 3),
            round((y_pred == y_test).sum()/len(y_pred), 3 )))

In [34]:
for n_est in [150, 200]:
    for max_depth in [5, 7, 9]:
        for lr in [1, 1.5]:
            train_GB(n_est, max_depth, lr)

Est: 150 / Depth: 5 / LR: 1 ----- Precision: 0.707 / Recall: 0.58 / Accuracy: 0.67
Est: 150 / Depth: 5 / LR: 1.5 ----- Precision: 0.699 / Recall: 0.58 / Accuracy: 0.665
Est: 150 / Depth: 7 / LR: 1 ----- Precision: 0.714 / Recall: 0.6 / Accuracy: 0.68
Est: 150 / Depth: 7 / LR: 1.5 ----- Precision: 0.655 / Recall: 0.57 / Accuracy: 0.635
Est: 150 / Depth: 9 / LR: 1 ----- Precision: 0.716 / Recall: 0.58 / Accuracy: 0.675
Est: 150 / Depth: 9 / LR: 1.5 ----- Precision: 0.756 / Recall: 0.62 / Accuracy: 0.71
Est: 200 / Depth: 5 / LR: 1 ----- Precision: 0.701 / Recall: 0.54 / Accuracy: 0.655
Est: 200 / Depth: 5 / LR: 1.5 ----- Precision: 0.701 / Recall: 0.61 / Accuracy: 0.675
Est: 200 / Depth: 7 / LR: 1 ----- Precision: 0.702 / Recall: 0.59 / Accuracy: 0.67
Est: 200 / Depth: 7 / LR: 1.5 ----- Precision: 0.738 / Recall: 0.62 / Accuracy: 0.7
Est: 200 / Depth: 9 / LR: 1 ----- Precision: 0.728 / Recall: 0.59 / Accuracy: 0.685
Est: 200 / Depth: 9 / LR: 1.5 ----- Precision: 0.744 / Recall: 0.61 / Acc

### XGBClassifier with OPTUNA

In [35]:
# XGBClassifier parameter search with optuna

def objective(trial):
    # Number of trees in random forest
    n_estimators = trial.suggest_int(name="n_estimators", low=10, high=500, step=10)

    min_child_weight = trial.suggest_int(name='min_child_weight', low=0, high=50, step=2) 

    # Maximum number of levels in tree
    max_depth = trial.suggest_int(name="max_depth", low=2, high=30, step=2)

    learning_rate = trial.suggest_loguniform(name="learning_rate", low=0.005, high=5)
    
    alpha = trial.suggest_int(name="alpha", low=0, high=10, step=1)

    #lambda =  trial.suggest_int(name="lambda", low=0, high=5, step=1)

    params = {
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        'min_child_weight':  min_child_weight,
        'alpha': alpha,
        #'lambda': lambda
    }
   
    model = XGBClassifier(random_state =42, **params)
    cv_score = cross_val_score(model, X_train1, y_train1, n_jobs=-1, cv=5)
    mean_cv_accuracy = cv_score.mean()
    return mean_cv_accuracy

study = optuna.create_study(direction='maximize')  # Adjust direction based on your optimization goal
study.optimize(objective, n_trials=5)

# Train a new model using the best parameters
best_model = XGBClassifier(**study.best_params)
best_model.fit(X_train1, y_train1)

y_pred = best_model.predict(X_test)
pred_train = best_model.predict(X_train1)


acc = round(accuracy_score(y_test, y_pred), 3)
acc_train = round(accuracy_score(y_train1, pred_train), 3)
precision, recall, f1, _ = score(y_test, y_pred, average='binary')
#overfit_acc = ((scores['train_accuracy']) - accuracy) * 100
overfitting = (acc_train-acc)*100 
print("Accuracy: {} || Precision: {} || Recall: {} || F1: {} || Overfit%: {}".format(acc, precision, recall, f1, overfitting ))
  
    

[I 2023-11-17 13:17:00,948] A new study created in memory with name: no-name-00ef6223-bc07-467d-b667-27f974ecdd10
/tmp/ipykernel_200/582357179.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform(name="learning_rate", low=0.005, high=5)
[I 2023-11-17 13:17:11,194] Trial 0 finished with value: 0.5475 and parameters: {'n_estimators': 410, 'min_child_weight': 44, 'max_depth': 4, 'learning_rate': 0.23695872855126765, 'alpha': 6}. Best is trial 0 with value: 0.5475.
/tmp/ipykernel_200/582357179.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform(name="learning_rate", low=0.005, high=5)
[I 2023

### CatBoostClassifier with OPTUNA

In [36]:
# CatBoostClassifier parameter search with optuna

def objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'iterations': trial.suggest_int('iterations', 50, 1000)
        }
    
    
    model = CatBoostClassifier(random_state =42, **params)
    
    cv_score = cross_val_score(model, X_train1, y_train1, n_jobs=-1, cv=5)
    mean_cv_accuracy = cv_score.mean()
    return mean_cv_accuracy

study = optuna.create_study(direction='maximize')  # Adjust direction based on your optimization goal
study.optimize(objective, n_trials=5)

# Train a new model using the best parameters
best_model = CatBoostClassifier(**study.best_params)
best_model.fit(X_train1, y_train1)

y_pred = best_model.predict(X_test)
pred_train = best_model.predict(X_train1)


acc = round(accuracy_score(y_test, y_pred), 3)
acc_train = round(accuracy_score(y_train1, pred_train), 3)
precision, recall, f1, _ = score(y_test, y_pred, average='binary')
#overfit_acc = ((scores['train_accuracy']) - accuracy) * 100
overfitting = (acc_train-acc)*100 
print("Accuracy: {} || Precision: {} || Recall: {} || F1: {} || Overfit%: {}".format(acc, precision, recall, f1, overfitting ))
    


183:	learn: 0.4124626	total: 675ms	remaining: 462ms
184:	learn: 0.4118110	total: 678ms	remaining: 458ms
185:	learn: 0.4107918	total: 681ms	remaining: 454ms
186:	learn: 0.4105517	total: 685ms	remaining: 450ms
187:	learn: 0.4100251	total: 688ms	remaining: 446ms
188:	learn: 0.4094880	total: 691ms	remaining: 443ms
189:	learn: 0.4080318	total: 695ms	remaining: 439ms
190:	learn: 0.4063828	total: 698ms	remaining: 435ms
191:	learn: 0.4053216	total: 702ms	remaining: 431ms
192:	learn: 0.4046829	total: 706ms	remaining: 428ms
193:	learn: 0.4041669	total: 712ms	remaining: 426ms
194:	learn: 0.4033764	total: 716ms	remaining: 422ms
195:	learn: 0.4029831	total: 720ms	remaining: 419ms
196:	learn: 0.4024387	total: 723ms	remaining: 415ms
197:	learn: 0.4017534	total: 726ms	remaining: 411ms
198:	learn: 0.4001781	total: 730ms	remaining: 407ms
199:	learn: 0.3993953	total: 733ms	remaining: 403ms
200:	learn: 0.3983252	total: 736ms	remaining: 399ms
201:	learn: 0.3977655	total: 739ms	remaining: 395ms
202:	learn: 

### RandomForestClassifier with OPTUNA

In [37]:
# RFC parameter search with optuna

def objective(trial):
    # Number of trees in random forest
    n_estimators = trial.suggest_int(name="n_estimators", low=10, high=200, step=10)

    # Maximum number of levels in tree
    max_depth = trial.suggest_int(name="max_depth", low=10, high=50, step=5)

    # Minimum number of samples required to split a node
    min_samples_split = trial.suggest_int(name="min_samples_split", low=2, high=10, step=2)

    # Minimum number of samples required at each leaf node
    min_samples_leaf = trial.suggest_int(name="min_samples_leaf", low=1, high=4, step=1)
    
    params = {
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf
    }
    
    model = RandomForestClassifier(random_state =42, **params)
    
    cv_score = cross_val_score(model, X_train1, y_train1, n_jobs=-1, cv=5)
    mean_cv_accuracy = cv_score.mean()
    return mean_cv_accuracy

study = optuna.create_study()
study.optimize(objective, n_trials=5)

# Train a new model using the best parameters
best_model = RandomForestClassifier(**study.best_params)
best_model.fit(X_train1, y_train1)

y_pred = best_model.predict(X_test)
pred_train = best_model.predict(X_train1)


acc = round(accuracy_score(y_test, y_pred), 3)
acc_train = round(accuracy_score(y_train1, pred_train), 3)
precision, recall, f1, _ = score(y_test, y_pred, average='binary')
#overfit_acc = ((scores['train_accuracy']) - accuracy) * 100
overfitting = (acc_train-acc)*100 
print("Accuracy: {} || Precision: {} || Recall: {} || F1: {} || Overfit%: {}".format(acc, precision, recall, f1, overfitting ))


[I 2023-11-17 13:23:31,460] A new study created in memory with name: no-name-53a73478-809e-43e0-b06d-037651c5da39
[I 2023-11-17 13:23:32,121] Trial 0 finished with value: 0.7212500000000001 and parameters: {'n_estimators': 60, 'max_depth': 30, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.7212500000000001.
[I 2023-11-17 13:23:33,426] Trial 1 finished with value: 0.7087500000000001 and parameters: {'n_estimators': 140, 'max_depth': 45, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.7087500000000001.
[I 2023-11-17 13:23:35,066] Trial 2 finished with value: 0.7037500000000001 and parameters: {'n_estimators': 180, 'max_depth': 30, 'min_samples_split': 6, 'min_samples_leaf': 4}. Best is trial 2 with value: 0.7037500000000001.
[I 2023-11-17 13:23:36,287] Trial 3 finished with value: 0.695 and parameters: {'n_estimators': 160, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 3 with value: 0.695.
[I 20

### GradientBoostingClassifier with Optuna

In [38]:
def objective(trial):
    # Number of trees in random forest
    n_estimators = trial.suggest_int(name="n_estimators", low=10, high=200, step=10)



    # Maximum number of levels in tree
    max_depth = trial.suggest_int(name="max_depth", low=10, high=50, step=5)

    # Minimum number of samples required to split a node
    min_samples_split = trial.suggest_int(name="min_samples_split", low=2, high=10, step=2)

    # Minimum number of samples required at each leaf node
    min_samples_leaf = trial.suggest_int(name="min_samples_leaf", low=1, high=4, step=1)
    
    params = {
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf
    }
    
    model = GradientBoostingClassifier(random_state =42, **params)
    
    cv_score = cross_val_score(model, X_train1, y_train1, n_jobs=-1, cv=5)
    mean_cv_accuracy = cv_score.mean()
    return mean_cv_accuracy

study = optuna.create_study()
study.optimize(objective, n_trials=5)

# Train a new model using the best parameters
best_model = GradientBoostingClassifier(**study.best_params)
best_model.fit(X_train1, y_train1)

y_pred = best_model.predict(X_test)
pred_train = best_model.predict(X_train1)


acc = round(accuracy_score(y_test, y_pred), 3)
acc_train = round(accuracy_score(y_train1, pred_train), 3)
precision, recall, f1, _ = score(y_test, y_pred, average='binary')
#overfit_acc = ((scores['train_accuracy']) - accuracy) * 100
overfitting = (acc_train-acc)*100 
print("Accuracy: {} || Precision: {} || Recall: {} || F1: {} || Overfit%: {}".format(acc, precision, recall, f1, overfitting ))


[I 2023-11-17 13:23:37,742] A new study created in memory with name: no-name-a645709e-c23d-4e29-9383-2e4f0f03bc25
[I 2023-11-17 13:23:40,530] Trial 0 finished with value: 0.7137500000000001 and parameters: {'n_estimators': 20, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.7137500000000001.
[I 2023-11-17 13:24:32,819] Trial 1 finished with value: 0.6824999999999999 and parameters: {'n_estimators': 180, 'max_depth': 45, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.6824999999999999.
[I 2023-11-17 13:24:57,746] Trial 2 finished with value: 0.68875 and parameters: {'n_estimators': 100, 'max_depth': 40, 'min_samples_split': 8, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.6824999999999999.
[I 2023-11-17 13:25:15,117] Trial 3 finished with value: 0.70375 and parameters: {'n_estimators': 80, 'max_depth': 30, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.6824999999999999.
[I

### RandomForestClassifier with Gridsearch

In [39]:
param = {'n_estimators': [190, 200, 300],
         'max_depth': [25, 35, 45],
         "min_samples_split": [4],
        "min_samples_leaf": [4]
         }

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_train1, y_train1)
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,0.469851,0.004066,0.038106,0.002042,25,4,4,300,"{'max_depth': 25, 'min_samples_leaf': 4, 'min_...",0.70625,0.74375,0.72500,0.71875,0.68125,0.71500,0.020767,1
5,0.496067,0.017499,0.034651,0.002055,35,4,4,300,"{'max_depth': 35, 'min_samples_leaf': 4, 'min_...",0.70000,0.73125,0.73750,0.72500,0.67500,0.71375,0.023184,2
7,0.347166,0.014474,0.028830,0.004602,45,4,4,200,"{'max_depth': 45, 'min_samples_leaf': 4, 'min_...",0.68125,0.72500,0.74375,0.73125,0.68750,0.71375,0.024812,3
8,0.510323,0.014886,0.035347,0.001897,45,4,4,300,"{'max_depth': 45, 'min_samples_leaf': 4, 'min_...",0.69375,0.73125,0.73750,0.72500,0.68125,0.71375,0.022150,3
0,0.304127,0.003421,0.029334,0.002678,25,4,4,190,"{'max_depth': 25, 'min_samples_leaf': 4, 'min_...",0.68750,0.73750,0.74375,0.71875,0.67500,0.71250,0.027099,5


### GradientBoostingClassifier with Gridsearch

In [40]:

param_gbc = {'n_estimators': [10, 50, 150],
         'max_depth': [10, 30], # depth of each tree, Nonr means that model can choose by default
        "min_samples_split": [5],
        "learning_rate": [1, 0.25]}

gs = GridSearchCV(gbc, param_gbc, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_train1, y_train1)
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6,0.174540,0.011285,0.009171,0.001861,0.25,10,5,10,"{'learning_rate': 0.25, 'max_depth': 10, 'min_...",0.70625,0.68125,0.71250,0.74375,0.71250,0.71125,0.019922,1
9,0.398467,0.014857,0.008191,0.000323,0.25,30,5,10,"{'learning_rate': 0.25, 'max_depth': 30, 'min_...",0.71250,0.65000,0.69375,0.75000,0.70000,0.70125,0.032210,2
8,2.543062,0.058148,0.011473,0.002557,0.25,10,5,150,"{'learning_rate': 0.25, 'max_depth': 10, 'min_...",0.67500,0.68750,0.68750,0.75000,0.70000,0.70000,0.026220,3
10,2.346624,0.045500,0.008717,0.000349,0.25,30,5,50,"{'learning_rate': 0.25, 'max_depth': 30, 'min_...",0.69375,0.67500,0.66250,0.75625,0.71250,0.70000,0.032835,3
7,0.883409,0.021997,0.008813,0.001589,0.25,10,5,50,"{'learning_rate': 0.25, 'max_depth': 10, 'min_...",0.68750,0.66250,0.68750,0.76250,0.69375,0.69875,0.033634,5


### XGBClassifier with GridSearch

In [41]:

param = {'n_estimators': [20, 440, 500, 550],
        'max_depth': [8, 20, 25], # depth of each tree, Nonr means that model can choose by default
        "min_child_weight": [2, 30, 45],
        "learning_rate": [0.57, 1.5, 2]}

gs = GridSearchCV(xgb, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_train1, y_train1)
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_min_child_weight,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.179872,0.006347,0.032821,0.004704,0.57,8,2,20,"{'learning_rate': 0.57, 'max_depth': 8, 'min_c...",0.68125,0.67500,0.71250,0.71250,0.67500,0.69125,0.017500,1
24,0.210352,0.007977,0.033052,0.002051,0.57,25,2,20,"{'learning_rate': 0.57, 'max_depth': 25, 'min_...",0.70000,0.69375,0.69375,0.70000,0.65000,0.68750,0.018957,2
13,2.269580,0.033595,0.038070,0.005377,0.57,20,2,440,"{'learning_rate': 0.57, 'max_depth': 20, 'min_...",0.72500,0.68750,0.65625,0.68750,0.66875,0.68500,0.023251,3
14,2.627536,0.081608,0.033663,0.004731,0.57,20,2,500,"{'learning_rate': 0.57, 'max_depth': 20, 'min_...",0.71875,0.68125,0.65625,0.68750,0.66250,0.68125,0.022009,4
12,0.208723,0.007580,0.031949,0.002406,0.57,20,2,20,"{'learning_rate': 0.57, 'max_depth': 20, 'min_...",0.68750,0.66875,0.67500,0.69375,0.67500,0.68000,0.009186,5


## Resume of best model selection 

In [84]:
models_final = [
    RandomForestClassifier(n_estimators=50, random_state= 42),
    RandomForestClassifier(n_estimators= 50, n_jobs=-1, max_depth=20), 
    RandomForestClassifier(n_estimators=30, max_depth=50, min_samples_split=6, min_samples_leaf=4),
    RandomForestClassifier(n_estimators=300, max_depth=35, min_samples_split=4, min_samples_leaf=4),
    RandomForestClassifier(n_estimators=190, max_depth=25, min_samples_split=4, min_samples_leaf=4),
    GradientBoostingClassifier(n_estimators=50),
    GradientBoostingClassifier(n_estimators=150, max_depth=7, learning_rate=1),
    GradientBoostingClassifier(n_estimators=10, max_depth=30, learning_rate=0.25, min_samples_split=5),
    GradientBoostingClassifier(n_estimators=10, max_depth=10, learning_rate=0.25, min_samples_split=5) 
    #XGBClassifier(n_estimators=420, min_child_weight=2, max_depth=2, learning_rate=0.1667, alpha=0,random_state= 42),
    #CatBoostClassifier(random_state =42)
]

In [85]:
def evaluate_models(models_final, X_train, y_train):
    skf = StratifiedKFold(n_splits=5)

    
    results = []

    for i, model in enumerate(models_final):
        model.fit(X_train, y_train)
        scores = cross_val_score(model, X_train, y_train.values.ravel(), cv=skf)

        y_pred = model.predict(X_test)
        accuracy = round(accuracy_score(y_test, y_pred), 3)
        scores = score(y_test, y_pred)
        mean_recall = round(scores[1].mean(), 3)
        mean_precision = round(scores[0].mean(), 3)
        mean_f1 = round(scores[2].mean(),3)

        train_pred = model.predict(X_train)
        acc_train = accuracy_score(y_train,train_pred)
        overfitting = (acc_train-accuracy)*100 

        # Append the results
        model_metrics = {'Model': model,
                        'Model score': scores,
                        'Accuracy': accuracy, 
                        'Precision': mean_precision,
                        'Recall': mean_recall,
                        'F1': mean_f1,
                        'Overfit%': overfitting}
        results.append(model_metrics)
          
     # Create a DataFrame from the results list
    results_df = pd.DataFrame(results)

    return results_df

In [86]:
results_df= evaluate_models(models_final, X_train1, y_train1)

In [87]:
results_df

,Model,Model score,Accuracy,Precision,Recall,F1,Overfit%
0,"(DecisionTreeClassifier(max_features='sqrt', r...","([0.6935483870967742, 0.7631578947368421], [0....",0.720,0.728,0.716,0.714,27.125
1,"(DecisionTreeClassifier(max_depth=20, max_feat...","([0.6691176470588235, 0.796875], [0.875, 0.531...",0.710,0.733,0.703,0.698,16.250
2,"(DecisionTreeClassifier(max_depth=50, max_feat...","([0.6893939393939394, 0.8088235294117647], [0....",0.730,0.749,0.724,0.721,10.500
3,"(DecisionTreeClassifier(max_depth=35, max_feat...","([0.6744186046511628, 0.7605633802816901], [0....",0.705,0.717,0.700,0.697,14.625
4,"(DecisionTreeClassifier(max_depth=25, max_feat...","([0.676923076923077, 0.7714285714285715], [0.8...",0.710,0.724,0.704,0.701,13.250
5,([DecisionTreeRegressor(criterion='friedman_ms...,"([0.6533333333333333, 0.88], [0.94230769230769...",0.710,0.767,0.700,0.687,10.500
6,([DecisionTreeRegressor(criterion='friedman_ms...,"([0.6754385964912281, 0.686046511627907], [0.7...",0.680,0.681,0.677,0.677,31.250
7,([DecisionTreeRegressor(criterion='friedman_ms...,"([0.6587301587301587, 0.7162162162162162], [0....",0.680,0.687,0.675,0.673,31.250
8,([DecisionTreeRegressor(criterion='friedman_ms...,"([0.6595744680851063, 0.8135593220338984], [0....",0.705,0.737,0.697,0.689,18.750


## Best model - RandomForestClassifier

In [88]:
# Find the row with the minimum 'Overfit%' value (best model)
best_model_row = results_df.loc[results_df['Overfit%'].idxmin()]

# Print the results of the best model
# Print best model
#result = tabulate(best_model_row), tablefmt='rounded_grid', showindex=True)
result = tabulate(pd.DataFrame(best_model_row), headers='keys', tablefmt='rounded_grid', showindex=True)
print(result)

╭─────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│             │ 2                                                                                                                      │
├─────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Model       │ RandomForestClassifier(max_depth=50, min_samples_leaf=4, min_samples_split=6,                                          │
│             │                        n_estimators=30)                                                                                │
├─────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Model score │ (array([0.68939394, 0.80882353]), array([0.875     , 0.57291667]), array([0.77118644, 0.67073171]), array([104,  96])) │
├─────────────┼──────────────────────────

In [89]:
rf_best = RandomForestClassifier(max_depth=50, min_samples_leaf=4, min_samples_split=6, n_estimators=30)
model_trained = rf_best.fit(X_train, y_train)
model_trained

RandomForestClassifier(max_depth=50, min_samples_leaf=4, min_samples_split=6,
                       n_estimators=30)

In [91]:
# Create pipeline that includes preprocessing and model
pipeline = Pipeline(steps =[('preprocessor', preprocessor),('model', model_trained)])
pipeline

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('clean',
                                  FunctionTransformer(func=<function clean_text at 0x7fedd13f5a60>)),
                                 ('tfv', TfidfVectorizer())])),
                ('model',
                 RandomForestClassifier(max_depth=50, min_samples_leaf=4,
                                        min_samples_split=6,
                                        n_estimators=30))])

In [92]:
#saving pipeline
with open('pipeline.pkl','wb')as file:
    pickle.dump(pipeline, file)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d8103776-8bc3-42d5-af3b-9ad38442056c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>